In [90]:
import csv
from tqdm import tqdm
import json 
import openai
import os
import numpy as np

def file_loader(name):
    set_hold = set()
    if name.endswith(".csv"):
        with open(name, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                next(csvreader)

                for row in csvreader:
                    set_hold.update(row[1:5])
    else:           
        data = np.load(name, allow_pickle=True)
        for item in data:
            set_hold.update(item['choice_list'][0:3])
        
    return set_hold

In [91]:
set_hold = file_loader('./WP-train.npy')

In [92]:
combined_data = []
directory_path = './'

for filename in os.listdir(directory_path):
    if filename.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            combined_data.extend(data)

In [93]:
for item in combined_data:
    key = list(item.keys())[0]
    if key in set_hold:
        set_hold.remove(key)

In [94]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [95]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [96]:
def extender(text):
    try:
        prompt = f"""What it the definition of "{text}"? write in in super large sentence."""
        response = get_completion(prompt)
        return({text:response})
    except:
        return text


In [97]:
hold = []
import concurrent.futures
with concurrent.futures.ProcessPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(extender, parameter) for parameter in set_hold]
    for future in tqdm(concurrent.futures.as_completed(futures),total=len(futures)):
        result = future.result(timeout=10)
        hold.append(result)

0it [00:00, ?it/s]


In [89]:
hold_json = []
hold_failed = []
for item in hold:
    if isinstance(item, dict):
        hold_json.append(item)
    else:
        hold_failed.append(item)

file_path = 'meaning.json'        
if os.path.exists(file_path):
    with open(file_path, 'r') as json_file:
        existing_data = json.load(json_file)
else:
    existing_data = []

existing_data.extend(hold_json)    
with open(file_path, 'w') as json_file:
    json.dump(existing_data, json_file)